### LICENSE PLATE OBJECT DETECTION

Based on https://huggingface.co/datasets/keremberke/license-plate-object-detection

In [17]:
# Install dependencies and libreries
!pip install torchmetrics
!pip install --upgrade --no-cache-dir gdown
!jupyter nbextension enable --py widgetsnbextension

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

optional arguments:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: kernel kernelspec migrate migrate.exe run script.py
troubleshoot troubleshoot.exe trust-script.py trust.exe

Jupyter command `jupyter-nbextension` not found.


In [18]:
# Import Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import os

import torch
import torchvision
import torchsummary
import torchmetrics
from torch.utils.tensorboard import SummaryWriter
from transformers import AutoFeatureExtractor
from datasets import load_dataset
from PIL import Image, ImageDraw
import torchvision.transforms as transform
# For data augmentation
from torchvision.transforms import Compose, Normalize, RandomResizedCrop, ColorJitter, ToTensor


In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [20]:
torch.backends.cudnn.benchmark = True

In [21]:

data_set = load_dataset( "keremberke/license-plate-object-detection", name="full")
data_set

DatasetDict({
    train: Dataset({
        features: ['image_id', 'image', 'width', 'height', 'objects'],
        num_rows: 6176
    })
    validation: Dataset({
        features: ['image_id', 'image', 'width', 'height', 'objects'],
        num_rows: 1765
    })
    test: Dataset({
        features: ['image_id', 'image', 'width', 'height', 'objects'],
        num_rows: 882
    })
})

In [22]:
# Test dataset features
ds_test = data_set['test']
ds_test.features

{'image_id': Value(dtype='int64', id=None),
 'image': Image(decode=True, id=None),
 'width': Value(dtype='int32', id=None),
 'height': Value(dtype='int32', id=None),
 'objects': Sequence(feature={'id': Value(dtype='int64', id=None), 'area': Value(dtype='int64', id=None), 'bbox': Sequence(feature=Value(dtype='float32', id=None), length=4, id=None), 'category': ClassLabel(names=['license_plate'], id=None)}, length=-1, id=None)}

In [23]:
# Train Dataset features
ds_train=data_set['train']
ds_train.features

{'image_id': Value(dtype='int64', id=None),
 'image': Image(decode=True, id=None),
 'width': Value(dtype='int32', id=None),
 'height': Value(dtype='int32', id=None),
 'objects': Sequence(feature={'id': Value(dtype='int64', id=None), 'area': Value(dtype='int64', id=None), 'bbox': Sequence(feature=Value(dtype='float32', id=None), length=4, id=None), 'category': ClassLabel(names=['license_plate'], id=None)}, length=-1, id=None)}

In [24]:
objects = data_set['train'][::]['objects']

: 

In [ ]:
objects[0]

### Preprocessing data

In [ ]:
#List of images
images = ds_train[::]['image']
#images

In [ ]:
# Showing the three first images
images[:3]

In [ ]:
# Preprocessing data
ANCHO_IMAGENES = 150
ALTO_IMAGENES = 150

tranform_ds = torchvision.transforms.Compose(
    [torchvision.transforms.Resize(size=(ANCHO_IMAGENES, ALTO_IMAGENES)),
    torchvision.transforms.RandomHorizontalFlip(0.5),
    torchvision.transforms.RandomResizedCrop(size=(64, 64)),
    torchvision.transforms.RandomRotation(degrees=90), torchvision.transforms.RandomPerspective(0.8),
    torchvision.transforms.RandomGrayscale(0.5),
    torchvision.transforms.ToTensor()])

In [ ]:
list_image_transformed = []
for images_tr in images:
    img_tr=tranform_ds(images_tr)
    list_image_transformed.append(img_tr)

In [ ]:
# Checking the data augmentation applied
img =list_image_transformed[0]

In [ ]:
# Transform the tensor to image
transform_img = transform.ToPILImage()

In [ ]:
image = transform_img(img)
image.show()

## Exportar imágenes de placas

In [54]:
import os

carpetas = ["train", "validation", "test"]
for carpeta in carpetas:
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

for ds_name in carpetas:
    images = data_set[ds_name][::]['image']
    objects = data_set[ds_name][::]['objects']
    for img, obj in zip(images, objects):
        if len(obj['id']) == 1:
            x, y, w, h = map(int, obj['bbox'][0])
            cropped_img = img.crop((x, y, x + w, y + h))
            cropped_img.save(f"{ds_name}/{obj['id'][0]}.jpg")